# Connect to ADLS Gen2 (Storage Account)

# Load Silver Parquet (Books & Authors)


In [0]:
# Load the books dataset from the silver (processed) layer
books = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/processed/books/"
)

# Load the authors dataset from the silver (processed) layer
authors = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/processed/authors/"
)

# Display first few records to confirm successful load
books.show(5)
authors.show(5)

# Print the schema of each dataset
books.printSchema()
authors.printSchema()


+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+---------+----+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|author_id|role|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+---------+----+--------------------+-

# Load Reviews Data

In [0]:
# -----------------------------------------------
#  Read compressed reviews JSON directly (gzip)
# -----------------------------------------------
from pyspark.sql import SparkSession

# Read directly from your ADLS Gen2 storage (gzip file)
reviews = spark.read \
    .option("mode", "PERMISSIVE") \
    .option("compression", "gzip") \
    .option("multiline", "true") \
    .json("abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/raw/reviews/goodreads_reviews_dedup.json.gz")

# -----------------------------------------------
#  Quick check
# -----------------------------------------------
reviews.printSchema()
reviews.show(5, truncate=False)
print(f"Total rows before cleaning: {reviews.count()}")


root
 |-- book_id: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- date_updated: string (nullable = true)
 |-- n_comments: long (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- rating: long (nullable = true)
 |-- read_at: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- started_at: string (nullable = true)
 |-- user_id: string (nullable = true)

+--------+------------------------------+------------------------------+----------+-------+------+------------------------------+--------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import col, length, trim

# Read raw (possibly malformed) reviews safely using permissive mode
reviews = spark.read.option("multiline", "true") \
    .option("compression", "gzip") \
    .option("mode", "PERMISSIVE") \
    .json("abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/raw/reviews/")

# Peek at some rows and schema
reviews.show(5, truncate=False)
reviews.printSchema()


+--------+------------------------------+------------------------------+----------+-------+------+------------------------------+--------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# coverted reviews data to parquet 

In [0]:

reviews.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/processed/reviews/"
)



# Read reviews (Parquet) and profile

In [0]:
from pyspark.sql.functions import col, length, trim, count, when

# Read raw (uncleaned) reviews from your processed (silver) layer
reviews_parquet = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/processed/reviews/"
)

# peek at rows and schema
reviews_parquet.show(5, truncate=False)
reviews_parquet.printSchema()

# Basic profiling: counts and potential issues
total_rows = reviews_parquet.count()
null_review_id = reviews_parquet.filter(col("review_id").isNull()).count()
null_book_id = reviews_parquet.filter(col("book_id").isNull()).count()
null_user_id = reviews_parquet.filter(col("user_id").isNull()).count()
null_rating = reviews_parquet.filter(col("rating").isNull()).count()
empty_text = reviews_parquet.filter(
    (col("review_text").isNull()) | (trim(col("review_text")) == "")
).count()

print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id: {null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")


+--------+------------------------------+------------------------------+----------+-------+------+------------------------------+--------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Clean reviews by removing problematic rows

In [0]:
from pyspark.sql.functions import col, trim, length

# ✅ Start from your Parquet-loaded DataFrame
df = reviews_parquet

# ------------------------------------------------
# 1 Drop rows missing critical keys
# ------------------------------------------------
df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

# ------------------------------------------------
# 2️ Enforce rating to be integer in [1..5]
# ------------------------------------------------
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)

# ------------------------------------------------
# 3️ Normalize text; drop empty or ultra-short reviews (<10 chars)
# ------------------------------------------------
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)

# ------------------------------------------------
# 4️ De-duplicate by review_id
# ------------------------------------------------
df = df.dropDuplicates(["review_id"])

# ------------------------------------------------
# 5️ Select final columns
# ------------------------------------------------
reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text",
    "date_added"
)

#  Preview the cleaned data
reviews_clean.show(5, truncate=False)
print(f"Cleaned dataset has {reviews_clean.count()} rows.")


+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Persist Cleaned Reviews

In [0]:
#  Write the cleaned reviews back to the silver (processed) layer
reviews_clean.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/processed/reviews/"
)

# Sanity check: re-read from disk and inspect schema + sample rows
reviews_verified = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/processed/reviews/"
)

reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)

print(f"Written cleaned rows: {reviews_verified.count()}")


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+--------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#  HOMEWORK 1

In [0]:
# Read from your processed (silver) zone
books = spark.read.parquet("abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/processed/books/")
authors = spark.read.parquet("abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/processed/authors/")
reviews_clean = spark.read.parquet("abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/processed/reviews/")


In [0]:
# Create the bridge table (book_authors)
book_authors = books.select("book_id", "author_id").distinct()

# Show schema and sample
book_authors.printSchema()
book_authors.show(5, truncate=False)


root
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)

+--------+---------+
|book_id |author_id|
+--------+---------+
|4541271 |16667    |
|35452242|16209952 |
|11890850|4999210  |
|11497621|8715     |
|28436751|1217665  |
+--------+---------+
only showing top 5 rows


In [0]:
from pyspark.sql import functions as F

curated_gold_table = reviews_clean \
    .join(books, reviews_clean.book_id == books.book_id, "inner") \
    .join(authors, books.author_id == authors.author_id, "inner") \
    .select(
        reviews_clean.review_id,
        reviews_clean.book_id,
        books.title,
        authors.author_id,
        authors.name.alias("author_name"),
        reviews_clean.user_id,
        reviews_clean.rating,
        reviews_clean.review_text,
        books.language_code,
        books.ratings_count.alias("n_votes"),
        reviews_clean.date_added 
    )

# Check schema and preview
curated_gold_table.printSchema()
curated_gold_table.show(5, truncate=False)



root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: string (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+--------+-------------------------------------------------------------+---------+-----------------+--------------------------------+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Write curated gold table to the Gold zone as Delta
curated_gold_table.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/gold/curated_reviews/"
)

# Register directly in Spark (no external location required)
spark.sql("DROP TABLE IF EXISTS curated_reviews")

spark.sql("""
CREATE TABLE curated_reviews
USING DELTA
AS SELECT * FROM delta.`abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/gold/curated_reviews/`
""")

# Confirm registration and preview
spark.sql("SHOW TABLES").show(truncate=False)

#  Verify the table content
spark.sql("""
SELECT author_name, title, rating, language_code
FROM curated_reviews
ORDER BY rating DESC
LIMIT 5
""").show(truncate=False)


+--------+---------------+-----------+
|database|tableName      |isTemporary|
+--------+---------------+-----------+
|default |curated_reviews|false      |
+--------+---------------+-----------+

+--------------+--------------------------------------------------------------------------+------+-------------+
|author_name   |title                                                                     |rating|language_code|
+--------------+--------------------------------------------------------------------------+------+-------------+
|Colleen Hoover|Ugly Love                                                                 |5     |eng          |
|Yann Martel   |Life of Pi                                                                |5     |             |
|Alexis Abbott |Path to Ruin (The Dangerous Men #3)                                       |5     |             |
|Erica Pike    |Black Hurricane (Boston Boys, #3)                                         |5     |eng          |
|Cris Hammond